In [ ]:
import json
from datetime import date

import petl as etl
from rap_utils.petl.io.parquet_view import ParquetView
from utils.paths import PROCESSED, SITE

## Website traffic

In [ ]:
WEBSITE = SITE / 'insights/digital/_data/website'
WEBSITE.mkdir(exist_ok=True, parents=True)

In [ ]:
(
    ParquetView(PROCESSED / 'combined/programme.parquet')
    .selecteq('project_name', 'Bradford 2025 Website')
    .selecteq('source', 'Website stats')
    .selecteq('variable', 'interactions')
    .cut('month', 'value')
    .rename('value', 'count')
    .addfieldusingcontext('cumulative', lambda p, c, n: (p.cumulative if p else 0) + c['count'])
    .convert('month', str)
    .tojson(WEBSITE / 'total_users.json', indent=2)
)

## Followers

In [ ]:
FOLLOWERS = SITE / 'insights/digital/_data/followers'
FOLLOWERS.mkdir(exist_ok=True, parents=True)

In [ ]:
followers = (
    etl.fromcsv(PROCESSED / 'digital/followers.csv')
    # .convert('Date', date.fromisoformat)
    .convertnumbers()
)

In [ ]:
with open(FOLLOWERS / 'latest.json', 'w', encoding='utf-8') as f: 
    json.dump(
        dict(
            followers
            .groupselectlast('variable')
            .addfield('data', lambda r: { 'count': r.value, 'updated': r.Date })
            .cut('variable', 'data')
            .records()
        ),
        f,
        indent=2,
    )

In [ ]:
(
    followers
    .recast()
    .tojson(FOLLOWERS / 'by_date.json', indent=2)
)